In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jun 29 15:18:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!ls

sample_data


In [ ]:
!pip install datasets==1.4.1
!pip install transformers==4.4.0
!pip install torchaudio
!pip install librosa
!pip install jiwer

     |████████████████████████████████| 194kB 12.7MB/s 
     |████████████████████████████████| 245kB 22.0MB/s 
     |████████████████████████████████| 122kB 30.6MB/s 
     |████████████████████████████████| 2.1MB 13.1MB/s 
     |████████████████████████████████| 901kB 49.6MB/s 
     |████████████████████████████████| 3.3MB 24.6MB/s 
     |████████████████████████████████| 1.9MB 12.4MB/s 
     |████████████████████████████████| 51kB 4.8MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149813 sha256=4128486d65650151fd59f8417d492287516869a4a0bc242cf91863da71512b55
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
from datasets import load_metric

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
%%bash
cp /content/drive/MyDrive/clipboard/meta.zip .
unzip meta.zip

Archive:  meta.zip
  inflating: dev_new.tsv             
  inflating: test_new.tsv            
  inflating: train_new.tsv           


In [ ]:
%%bash
cp /content/drive/MyDrive/clipboard/mp3.zip .
unzip mp3.zip

Archive:  mp3.zip
   creating: single_channel_mp3/
  inflating: single_channel_mp3/Kettle_20.mp3  
  inflating: single_channel_mp3/15_01-18.00_segment-6.mp3  
  inflating: single_channel_mp3/18_01-18ю00_segment-2.mp3  
  inflating: single_channel_mp3/20_01-18.00_segment-17.mp3  
  inflating: single_channel_mp3/17_12-18.00_segment-12.mp3  
  inflating: single_channel_mp3/A chatterbox and a wanton girl_7.mp3  
  inflating: single_channel_mp3/Brandy_12.mp3  
  inflating: single_channel_mp3/Larvae_5.mp3  
  inflating: single_channel_mp3/Water cart_32.mp3  
  inflating: single_channel_mp3/Faces_3.mp3  
  inflating: single_channel_mp3/Fishing with Ilya_7.mp3  
  inflating: single_channel_mp3/Having bear ears_17.mp3  
  inflating: single_channel_mp3/Being a child_2.mp3  
  inflating: single_channel_mp3/Chapaev_11.mp3  
  inflating: single_channel_mp3/11_01-18.00_segment-29.mp3  
  inflating: single_channel_mp3/B03___09_Luke________CKTIBTP1DA.mp3  
  inflating: single_channel_mp3/I am from Chu

In [ ]:
drive.flush_and_unmount()

In [ ]:
import pandas as pd
train = pd.concat([pd.read_csv('train_new.tsv', sep='\t')[['path', 'sentence']].iloc[:1422], pd.read_csv('train_new.tsv', sep='\t')[['path', 'sentence']].iloc[1423:]])
test = pd.read_csv('test_new.tsv', sep='\t')[['path', 'sentence']]
dev = pd.read_csv('dev_new.tsv', sep='\t')[['path', 'sentence']]


In [ ]:
train = pd.concat([train, dev]).reset_index(drop=True)

In [ ]:
train

,path,sentence
0,17_12-18.00_segment-13.wav,минкыри морыкы генанымнылявлекн авын аройвыгые...
1,Being a child_24.wav,ялгытык ялгыт ныялгынморэ нытарагӄэнатъым йъэл...
2,Wyttal_2.wav,рымавыттэʔм нинэнръэлевивым ынкы гэвтыт и лыгэ...
3,Muscovite woman_3.wav,игыт намӈылёгъан тумгэ
4,The day before yesterday_3.wav,ынкы ыʼтвиӈын гэетлин
...,...,...
1650,Water cart_28.wav,опопыӈа торгынан чинит ӄэнарэрыркынэтык йыӄӄэй...
1651,Breast_13.wav,ынӄэнэ рэнриԓьилгын ӄытэнкэвӈыгын
1652,27_01-18.00_segment-30.wav,ынанмыкысо гененелинет венкен нымытвалит мытли...
1653,A chatterbox and a wanton girl_3.wav,ӄол вэтгавӈавъым ӄол камэлгыӈав


In [ ]:
train.path = train.path.apply(lambda x: 'single_channel_mp3/' + x.split(".wav")[0] + ".mp3")
test.path = test.path.apply(lambda x: 'single_channel_mp3/' + x.split(".wav")[0] + ".mp3")

In [ ]:
from datasets import Dataset
train, test = Dataset.from_pandas(train), Dataset.from_pandas(test)

NameError: ignored

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(train.remove_columns(["path"]))

,sentence
0,ынӄо бригадиро гатвален
1,ӄол нымытваԓьа нырантомгавӄэнат фиԓьмыт пэгчиӈу нинэлгыӄинэт ылпыкалегыргыт ӄутынэ ым рытомгаво нэрэнтыӈын фиԓьм вагыргыгъет ымыԓьорыкы лыги тэкэлиӈыԓьин юрий рытгэвынин
2,и вот иʼныӈэ энмэна эн ынӄэн иʼны мэԓьыттъывытрынэ ныръэӄин ӄынут гэгиинрэлин ӄынутэ гытъэвыԓьын гэймэвлин оʼравэтԓьак
3,и это лгэкоргаквъатъым нанрагтатынат
4,гемелевлинет мытлинса мытлинккыликкин ниреккыликкин конасгынкен нымытвалит
5,еттык ӄыпалёмтэлгыткы торпыӈылтэ
6,пынлепы коронавируса гатылиноленат чукоткак ныраса мытлинккылеккен нереккыликкин кыликкин мынгыткен ораветлят ныроса мытлинккылеккен ныраккылеккен мынгыткен нырок гемелевлинет ныярамалявкенат ныраса мытлинккылеккен мынгыткен нерамытлинен нымытвалит ныраргаре гевьйитеԓьпылинет ынкена тагена
7,минкыри морыкы генанымнылявлен иненлелин милсертенмысыкен маргарита тнескина конкурсанветы генивылинет кыликкин гапароԓьма заявкат
8,ынӄоръым ԓьэԓьэнры ӄынвэт ԓьэленръугъиʔм
9,еттык кыпалёмтеԓьгыткы торпыныԓьте гыым энанымнылявылевым нина пенеԓькут


In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\;\:\"\“\%\‘\”\�\dža-zA-Z\n–́]'

def remove_special_characters(batch):
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["sentence"]).lower() + " "
    return batch

train = train.map(remove_special_characters)
test = test.map(remove_special_characters)

In [ ]:
show_random_elements(train.remove_columns(["path"]))

,sentence
0,тумгытумэ транӄамэтвавӈынъым
1,игыр ым ытри нимрыривлетӄинэт ныкимитъынэймэвӄинэт яама мэйӈывиԓьэт ковлёрвыт такси
2,тэлиӈ тэ тъаӄалеӈӈытоаʼкъым
3,ӈэлвыл ыргин уйӈэ гэнъэтлин
4,гатваленат тэӈылӄотъым эʼӄэлӄот
5,ынӈин ԓьэлеӈитгъи а ӈэргэргитгъи ԓьэлеӈитгъи
6,чукоткак гамасярантален тагенгыргын орви майнораветлят
7,акциянлянветы насынкенат нымытвалит округакен
8,ынкы гетенинайвыткулин ийам нынылин пувракыт венкен
9,ынкенайпы тыплиткуркын атав нымеԓьав


In [ ]:
test

Dataset({
    features: ['path', 'sentence'],
    num_rows: 185
})

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocab_train = train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train.column_names)
vocab_test = test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test.column_names)

In [ ]:
vocab_test

Dataset({
    features: ['all_text', 'vocab'],
    num_rows: 1
})

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
print(vocab_dict)


{'т': 0, 'ф': 1, 'о': 2, ' ': 3, 'ч': 4, 'у': 5, 'ʔ': 6, 'б': 7, 'ж': 8, 'с': 9, 'в': 10, 'е': 11, 'н': 12, 'п': 13, 'м': 14, 'з': 15, 'к': 16, 'ш': 17, 'ы': 18, 'ԓ': 19, 'ь': 20, 'ӈ': 21, 'ʼ': 22, 'ю': 23, 'й': 24, 'ъ': 25, 'х': 26, 'а': 27, 'ц': 28, 'г': 29, 'л': 30, 'и': 31, 'щ': 32, 'ӄ': 33, 'ё': 34, 'э': 35, 'р': 36, 'я': 37, 'д': 38}


In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

41


In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
!cat vocab.json

{"\u0442": 0, "\u0444": 1, "\u043e": 2, "\u0447": 4, "\u0443": 5, "\u0294": 6, "\u0431": 7, "\u0436": 8, "\u0441": 9, "\u0432": 10, "\u0435": 11, "\u043d": 12, "\u043f": 13, "\u043c": 14, "\u0437": 15, "\u043a": 16, "\u0448": 17, "\u044b": 18, "\u0513": 19, "\u044c": 20, "\u04c8": 21, "\u02bc": 22, "\u044e": 23, "\u0439": 24, "\u044a": 25, "\u0445": 26, "\u0430": 27, "\u0446": 28, "\u0433": 29, "\u043b": 30, "\u0438": 31, "\u0449": 32, "\u04c4": 33, "\u0451": 34, "\u044d": 35, "\u0440": 36, "\u044f": 37, "\u0434": 38, "|": 3, "[UNK]": 39, "[PAD]": 40}

In [ ]:
train_copy = train

In [ ]:
from transformers import Wav2Vec2CTCTokenizer, Wav2Vec2FeatureExtractor

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [ ]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1,
                                             sampling_rate=16000,
                                             padding_value=0.0,
                                             do_normalize=True,
                                             return_attention_mask=True)


In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [ ]:
!ls

dev_new.tsv  mp3.zip	  single_channel_mp3  train_new.tsv
meta.zip     sample_data  test_new.tsv	      vocab.json


In [ ]:
print(train[0])


{'path': 'single_channel_mp3/17_12-18.00_segment-13.mp3', 'sentence': 'минкыри морыкы генанымнылявлекн авын аройвыгыепытколин округакен роспотребнадзора антон семенихин яливену нереԓьгынынет инкун эвневыԓьтумекетке ораветлят '}


In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    try:
      speech_array, sampling_rate = torchaudio.load(batch["path"])
      batch["speech"] = speech_array[0].numpy()
      batch["sampling_rate"] = sampling_rate
      batch["target_text"] = batch["sentence"]
      return batch
    except RuntimeError:
      print(batch['path'])

train = train.map(speech_file_to_array_fn, remove_columns=train.column_names)
test = test.map(speech_file_to_array_fn, remove_columns=test.column_names)


In [ ]:
import librosa
import numpy as np
from struct import error

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 44_100, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

train = train.map(resample)
test = test.map(resample)

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train))

ipd.Audio(data=np.asarray(train[rand_int]["speech"]), autoplay=True, rate=16000)


In [ ]:
train = train.filter(function=lambda x: x['speech'].__len__() > 0)
test = test.filter(function=lambda x: x['speech'].__len__() > 0)

In [ ]:
rand_int = random.randint(0, len(train))

print("Target text:", train[rand_int]["target_text"])
print("Input array shape:", np.asarray(train[rand_int]["speech"]).shape)
print("Sampling rate:", train[rand_int]["sampling_rate"])

Target text: ытыёԓькенат нутин гивик ковлёрвы трэколыт венеты герелинет нырокавык ынкам ныракавык январек нерамытлинкавык етыля герелинет чукоткагты мынгыткен амнырокен тоннат тыневыёт нымытвалеты 
Input array shape: (269569,)
Sampling rate: 16000


In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

train = train.map(prepare_dataset, remove_columns=train.column_names, batch_size=8, num_proc=4, batched=True)
test = test.map(prepare_dataset, remove_columns=test.column_names, batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

Couldn't find file locally at cer/cer.py, or remotely at https://raw.githubusercontent.com/huggingface/datasets/1.4.1/metrics/cer/cer.py.
The file was picked from the master branch on github instead at https://raw.githubusercontent.com/huggingface/datasets/master/metrics/cer/cer.py.


In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}


In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53",
    attention_dropout=0.06,
    hidden_dropout=0.05,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)


Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="/content/drive/MyDrive/clipboard/trynna_train",
  # output_dir="./wav2vec2-large-xlsr-turkish-demo",
  group_by_length=True,
  per_device_train_batch_size=1,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=6,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train,
    eval_dataset=test,
    tokenizer=processor.feature_extractor,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/torch/optim

Step,Training Loss,Validation Loss,Wer,Cer,Runtime,Samples Per Second
400,inf,inf,1.000000,0.986060,39.589500,4.673000
800,nan,inf,1.000000,0.986060,39.163900,4.724000
1200,nan,inf,0.998362,0.870367,39.012900,4.742000
1600,nan,inf,0.990991,0.387145,39.122700,4.729000
2000,nan,inf,0.908272,0.295743,39.191200,4.720000
2400,nan,inf,0.880426,0.256260,39.247600,4.714000
2800,nan,inf,0.864046,0.240902,39.433000,4.692000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default be

Step,Training Loss,Validation Loss,Wer,Cer,Runtime,Samples Per Second
400,inf,inf,1.000000,0.986060,39.589500,4.673000
800,nan,inf,1.000000,0.986060,39.163900,4.724000
1200,nan,inf,0.998362,0.870367,39.012900,4.742000
1600,nan,inf,0.990991,0.387145,39.122700,4.729000
2000,nan,inf,0.908272,0.295743,39.191200,4.720000
2400,nan,inf,0.880426,0.256260,39.247600,4.714000
2800,nan,inf,0.864046,0.240902,39.433000,4.692000
3200,nan,inf,0.809992,0.219115,39.542700,4.678000
3600,nan,inf,0.801802,0.206260,39.118600,4.729000
4000,nan,inf,0.778870,0.200918,39.369000,4.699000


/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  self.args.max_grad_norm,
/usr/local/lib/python3.7/dist-packages/transformers/trainer.py:1083: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default be

TrainOutput(global_step=4962, training_loss=nan, metrics={'train_runtime': 4215.0636, 'train_samples_per_second': 1.177, 'total_flos': 1.8647323526456973e+18, 'epoch': 6.0, 'init_mem_cpu_alloc_delta': 1711554, 'init_mem_gpu_alloc_delta': 1261923328, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1789925, 'train_mem_gpu_alloc_delta': 3780844544, 'train_mem_cpu_peaked_delta': 79671131, 'train_mem_gpu_peaked_delta': 3519987200})

In [ ]:
import transformers

In [ ]:
transformers.PreTrainedModel.from_pretrained('/content/shmive/MyDrive/clipboard/pretrained/pytorch_model.bin')

AttributeError: ignored

In [ ]:
def compare(n, data):
  input_dict = processor(test["input_values"][n], return_tensors="pt", padding=True)
  logits = model(input_dict.input_values.to("cuda")).logits
  pred_ids = torch.argmax(logits, dim=-1)[0]
  print("Prediction:")
  print(processor.decode(pred_ids))
  print("\nReference:")
  print(data["sentence"][n].lower())


In [ ]:
test_transcription = pd.read_csv('test_new.tsv', sep='\t')[['path', 'sentence']]

In [ ]:
compare(10, test_transcription)

It is strongly recommended to pass the ``sampling_rate`` argument to this function.Failing to do so can result in silent errors that might be hard to debug.


Prediction:
ынивк

Reference:
нивӄин


In [ ]:
from google.colab import drive
drive.mount('/content/shmive')

Mounted at /content/shmive


In [ ]:
rm content/shmive/MyDrive/clipboard/pretrained -rf

In [ ]:
import Sentencepiece

ModuleNotFoundError: ignored

In [ ]:
from transformers import Wav2Vec2ForCTC

In [ ]:
transformers.PreTrainedModel.from_pretrained(pretrained_model_name_or_path='/content/shmive/MyDrive/clipboard/pretrained/')

AttributeError: ignored

In [ ]:
model.save_pretrained("/content/shmive/MyDrive/clipboard/pretrained")

In [ ]:
!mv /content/drive/MyDrive/clipboard/trynna_train/* /content/shmive/MyDrive/clipboard/trynna_train